In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os

# Set seed for reproducibility
torch.manual_seed(42)


In [ ]:
# Change path to one valid .npz file from your dataset
car_path = "outputs/pad_masked_slices/DrivAer_F_D_WM_WW_0005_axis-x.npz"
data = np.load(car_path)

slices = data["slices"]           # (80, 6500, 2)
point_mask = data["point_mask"]   # (80, 6500)
slice_mask = data["slice_mask"]   # (80,)

# Find the first real slice (non-zero in slice_mask)
slice_idx = int(np.argmax(slice_mask))

slice_points = slices[slice_idx]         # (6500, 2)
point_mask_slice = point_mask[slice_idx]  # (6500,)

print(f"Using slice #{slice_idx} with {int(point_mask_slice.sum())} valid points")


In [ ]:
plt.figure(figsize=(6, 6))
valid_points = slice_points[point_mask_slice.astype(bool)]
plt.scatter(valid_points[:, 0], valid_points[:, 1], s=1)
plt.title(f"Slice #{slice_idx} - (y, z) points")
plt.xlabel("y")
plt.ylabel("z")
plt.axis("equal")
plt.grid(True)
plt.show()


In [ ]:
class PointNet2D(nn.Module):
    def __init__(self, input_dim=2, emb_dim=128):
        super(PointNet2D, self).__init__()
        self.mlp = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=1),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=1),
            nn.ReLU(),
            nn.Conv1d(128, emb_dim, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x, mask=None):
        # x: (B, N, 2) → (B, 2, N)
        x = x.transpose(1, 2)
        features = self.mlp(x)  # (B, emb_dim, N)

        if mask is not None:
            mask = mask.unsqueeze(1)  # (B, 1, N)
            features = features * mask + (1 - mask) * (-1e9)  # safe masked max

        embedding = torch.max(features, dim=2)[0]  # (B, emb_dim)
        return embedding



In [ ]:
# Convert slice to tensor
slice_tensor = torch.tensor(slice_points, dtype=torch.float32).unsqueeze(0)  # (1, 6500, 2)
mask_tensor = torch.tensor(point_mask_slice, dtype=torch.float32).unsqueeze(0)  # (1, 6500)

# Create model
pointnet = PointNet2D(input_dim=2, emb_dim=128)

# Run forward pass
embedding = pointnet(slice_tensor, mask_tensor)

print("Output embedding shape:", embedding.shape)
print("Sample embedding:", embedding)  # show first 5 values


In [ ]:
# %%
import torch
import torch.nn as nn

class LSTMSliceEncoder(nn.Module):
    def __init__(self, input_dim=128, hidden_dim=128, num_layers=1, bidirectional=False):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,    # slice embedding size
            hidden_size=hidden_dim,  # output embedding per car
            num_layers=num_layers,
            batch_first=True,        # input shape: (B, S, D)
            bidirectional=bidirectional
        )
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim

    def forward(self, x):
        # x shape: (B, S, D) → (batch, 80 slices, 128 features)
        lstm_out, (h_n, c_n) = self.lstm(x)  # h_n: (num_layers × num_directions, B, H)

        # Get the last layer's hidden state
        if self.bidirectional:
            # concat forward + backward
            h_final = torch.cat((h_n[-2], h_n[-1]), dim=-1)  # (B, 2H)
        else:
            h_final = h_n[-1]  # (B, H)

        return h_final  # this becomes your car-level feature


In [ ]:
# %%
import torch.nn as nn

class CdRegressor(nn.Module):
    def __init__(self, input_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)  # (B,)


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset

class CarSlicesDataset(torch.utils.data.Dataset):
    def __init__(self, ids_txt, npz_dir, csv_path, max_cars=100):
        self.car_ids = [line.strip() for line in open(ids_txt)][:max_cars]
        self.npz_dir = npz_dir
        self.df = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.car_ids)

    def __getitem__(self, idx):
        car_id = self.car_ids[idx]
        path = os.path.join(self.npz_dir, f"{car_id}_axis-x.npz")
        data = np.load(path)

        slices = torch.tensor(data["slices"], dtype=torch.float32)          # (80, 6500, 2)
        point_mask = torch.tensor(data["point_mask"], dtype=torch.float32)  # (80, 6500)
        slice_mask = torch.tensor(data["slice_mask"], dtype=torch.float32)  # (80,)

        cd_value = self.df[self.df["Design"] == car_id]["Average Cd"].values[0]
        return slices, point_mask, slice_mask, torch.tensor(cd_value, dtype=torch.float32)


In [ ]:
class CdPredictorNet(nn.Module):
    def __init__(self, pointnet, lstm_encoder, regressor):
        super().__init__()
        self.pointnet = pointnet
        self.lstm_encoder = lstm_encoder
        self.regressor = regressor

    def forward(self, slices, point_mask, slice_mask):
        B, S, N, D = slices.shape  # (B, 80, 6500, 2)
        slices = slices.view(B * S, N, D)
        mask = point_mask.view(B * S, N)

        # Slice-wise feature extraction (batched)
        slice_emb = self.pointnet(slices, mask)  # (B×S, 128)
        slice_emb = slice_emb.view(B, S, -1)     # (B, 80, 128)

        car_emb = self.lstm_encoder(slice_emb)   # (B, 128 or 256)
        cd_pred = self.regressor(car_emb)        # (B,)

        return cd_pred


In [ ]:
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Instantiate model components
pointnet = PointNet2D(input_dim=2, emb_dim=128)
lstm_encoder = LSTMSliceEncoder(input_dim=128, hidden_dim=128, bidirectional=False)
regressor = CdRegressor(input_dim=128)

model = CdPredictorNet(pointnet, lstm_encoder, regressor)

device = torch.device("cpu")
model.to(device)

# Load dataset
train_dataset = CarSlicesDataset(
    ids_txt="data/subset_dir/train_design_ids.txt",
    npz_dir="outputs/pad_masked_slices",
    csv_path="data/DrivAerNetPlusPlus_Drag_8k_cleaned.csv",
    max_cars=100
)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)

# Optimizer & Loss
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for slices, point_mask, slice_mask, cd_gt in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        slices = slices.to(device)
        point_mask = point_mask.to(device)
        cd_gt = cd_gt.to(device)

        cd_pred = model(slices, point_mask, slice_mask)
        loss = loss_fn(cd_pred, cd_gt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * slices.size(0)

    avg_loss = epoch_loss / len(train_loader.dataset)
    print(f"✅ Epoch {epoch+1}: Avg MSE Loss = {avg_loss:.5f}")


In [ ]:
import numpy as np
import torch
import pandas as pd

# Load model
model.eval()

# Load full training IDs and select test subset (first 10 not used in training)
with open("data/subset_dir/train_design_ids.txt") as f:
    all_ids = [line.strip() for line in f]

test_ids = all_ids[:10]  # Pick 10 early cars

# Load CSV to get ground truth Cd
df = pd.read_csv("data/DrivAerNetPlusPlus_Drag_8k_cleaned.csv")

# Initialize storage
preds = []
trues = []

# Inference loop
for car_id in test_ids:
    path = f"outputs/pad_masked_slices/{car_id}_axis-x.npz"
    data = np.load(path)

    slices = torch.tensor(data["slices"], dtype=torch.float32).unsqueeze(0).to(device)
    point_mask = torch.tensor(data["point_mask"], dtype=torch.float32).unsqueeze(0).to(device)
    slice_mask = torch.tensor(data["slice_mask"], dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        cd_pred = model(slices, point_mask, slice_mask).item()
    
    cd_true = df[df["Design"] == car_id]["Average Cd"].values[0]

    preds.append(cd_pred)
    trues.append(cd_true)

    print(f"🚗 {car_id} → Predicted Cd: {cd_pred:.4f} | True Cd: {cd_true:.4f}")


In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(trues, preds)
print(f"\n📊 R² Score on 60-car test subset: {r2:.4f}")


In [ ]:
import numpy as np
import torch

# Load a test car not seen in training
car_id = "F_S_WWC_WM_498"  # or any from train_design_ids.txt[150:]

# Load and convert to tensors
data = np.load(f"outputs/pad_masked_slices/{car_id}_axis-x.npz")
slices = torch.tensor(data["slices"], dtype=torch.float32).unsqueeze(0).to(device)        # (1, 80, 6500, 2)
point_mask = torch.tensor(data["point_mask"], dtype=torch.float32).unsqueeze(0).to(device) # (1, 80, 6500)
slice_mask = torch.tensor(data["slice_mask"], dtype=torch.float32).unsqueeze(0).to(device) # (1, 80)

# Forward pass through full model
model.eval()
with torch.no_grad():
    print("🟦 Input shape:", slices.shape)

    # Step 1: Slice embeddings from PointNet2D
    B, S, N, D = slices.shape
    flat_slices = slices.view(B * S, N, D)
    flat_mask = point_mask.view(B * S, N)
    
    print("🟦 Flattened for PointNet2D:", flat_slices.shape)
    slice_emb = model.pointnet(flat_slices, flat_mask)   # (B×S, 128)
    print("🟦 Slice embeddings shape:", slice_emb.shape)

    slice_emb = slice_emb.view(B, S, -1)  # (1, 80, 128)
    print("🟦 Sequence ready for LSTM:", slice_emb.shape)

    # Step 2: LSTM
    car_feature = model.lstm_encoder(slice_emb)  # (1, 128)
    print("🟦 Output of LSTM (car embedding):", car_feature.shape)

    # Step 3: Cd prediction
    cd_pred = model.regressor(car_feature)
    print("🟩 Final Predicted Cd:", cd_pred.item())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# slice_emb: shape (80, 128), from your diagnostic pass
embedding_np = slice_emb.squeeze(0).cpu().detach().numpy()  # Now shape = (80, 128)

plt.figure(figsize=(12, 6))
sns.heatmap(embedding_np, cmap='viridis', cbar=True)
plt.title("🧠 Slice-wise Embeddings (80 slices × 128 features)")
plt.xlabel("Embedding Dimension")
plt.ylabel("Slice Index (front to rear)")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Make sure `slice_emb` is (80, 128) and on CPU
slice_emb_np = slice_emb.squeeze(0).cpu().detach().numpy()

# Run PCA
pca = PCA(n_components=2)
slice_pca = pca.fit_transform(slice_emb_np)  # shape: (80, 2)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(slice_pca[:, 0], slice_pca[:, 1], marker='o', linestyle='-', color='steelblue')
for i in range(0, 80, 10):
    plt.text(slice_pca[i, 0], slice_pca[i, 1], str(i), fontsize=8)

plt.title("🚗 PCA of Slice Embeddings (Front → Rear)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.tight_layout()
plt.show()
